In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# EasyOCR 사용

from ITT import (
    preprocess_image,           # 이미지 로드 (컬러 이미지 반환)
    ocr_with_structure,         # EasyOCR로 단어 단위 탐지
    group_by_line,              # 라인 단위 그룹화
    group_by_paragraph,         # 문단 단위 그룹화
    generate_text_from_data,    # 텍스트 생성
    call_llm_for_correction,    # OCR 교정 (LLM)
    call_llm_for_description,   # LLM 설명 생성
    save_output                 # 결과 저장
)

# 테스트용 이미지 경로
IMAGE_PATH = "img/BB.png"
print(f"테스트 이미지: {IMAGE_PATH}")


In [ ]:
# Step 1: preprocess_image 테스트
# preprocess_image는 원본 컬러 이미지를 반환합니다 (EasyOCR은 컬러 이미지도 처리 가능)
import cv2
import matplotlib.pyplot as plt

img = preprocess_image(IMAGE_PATH)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

print(f"이미지 shape: {img.shape}")
print(f"이미지 dtype: {img.dtype}")
print(f"채널 수: {img.shape[2] if len(img.shape) == 3 else 1}")

# 시각화
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.title("Preprocessed Image")
plt.imshow(img)
plt.axis('off')

# plt.subplot(1, 2, 2)
# plt.title("Image for OCR (RGB)")
# plt.imshow(img_rgb)
# plt.axis('off')

# plt.tight_layout()
# plt.show()


In [ ]:
# Step 2: ocr_with_structure 테스트 (EasyOCR 사용)
# 핵심: EasyOCR이 직접 단어 단위로 탐지! (한국어+영어 지원)
# 주의: 첫 실행 시 모델 다운로드가 필요할 수 있습니다 (1-2분 소요)

words_data = ocr_with_structure(img, level='word', langs=['ko', 'en'])

print(f"탐지된 단어 개수: {len(words_data)}")
print(f"\n처음 10개 단어:")
for i, w in enumerate(words_data[:10]):
    print(f"  [{i+1}] '{w['text']}' at ({w['x']}, {w['y']}) conf={w['conf']}% line={w.get('line_num', 'N/A')}")

# 시각화: 탐지된 단어 박스 표시
img_with_boxes = img_rgb.copy()
for w in words_data:
    cv2.rectangle(img_with_boxes, (w['x'], w['y']), (w['x']+w['w'], w['y']+w['h']), (255, 0, 0), 2)

plt.figure(figsize=(14, 8))
plt.title(f"Detected Words ({len(words_data)} words) - EasyOCR")
plt.imshow(img_with_boxes)
plt.axis('off')
plt.show()


In [ ]:


# Step 3: group_by_line 테스트 (라인 단위 그룹화)
lines_data = group_by_line(words_data)

print(f"그룹화된 라인 개수: {len(lines_data)}")
print(f"\n각 라인 내용 (처음 15개):")
for i, line in enumerate(lines_data[:15]):
    preview = line['text'][:60] + '...' if len(line['text']) > 60 else line['text']
    print(f"  Line {i+1} (line_num={line.get('line_num', 'N/A')}): '{preview}'")

if len(lines_data) > 15:
    print(f"  ... (총 {len(lines_data)}개 라인)")

# 시각화: 라인 박스 표시
img_with_lines = img_rgb.copy()
for line in lines_data:
    cv2.rectangle(img_with_lines, (line['x'], line['y']), 
                  (line['x']+line['w'], line['y']+line['h']), (0, 255, 0), 2)

plt.figure(figsize=(14, 8))
plt.title(f"Detected Lines ({len(lines_data)} lines)")
plt.imshow(img_with_lines)
plt.axis('off')
plt.show()


In [ ]:
# Step 4: group_by_paragraph 테스트 (문단 단위 그룹화)
# 라인 간격 기반으로 문단 구성 (평균 라인 높이의 1.5배 이상이면 새 문단)
paragraphs_data = group_by_paragraph(words_data, para_threshold_ratio=1.5)

print(f"그룹화된 문단 개수: {len(paragraphs_data)}")
print(f"\n각 문단 내용 (처음 10개):")
for i, para in enumerate(paragraphs_data[:10]):
    preview = para['text'][:80] + '...' if len(para['text']) > 80 else para['text']
    print(f"  Paragraph {i+1} (para_num={para.get('paragraph_num', 'N/A')}): '{preview}'")

if len(paragraphs_data) > 10:
    print(f"  ... (총 {len(paragraphs_data)}개 문단)")

# 시각화: 문단 박스 표시
img_with_paras = img_rgb.copy()
for para in paragraphs_data:
    cv2.rectangle(img_with_paras, (para['x'], para['y']), 
                  (para['x']+para['w'], para['y']+para['h']), (0, 0, 255), 3)

plt.figure(figsize=(14, 8))
plt.title(f"Detected Paragraphs ({len(paragraphs_data)} paragraphs)")
plt.imshow(img_with_paras)
plt.axis('off')
plt.show()


In [ ]:
# Step 5: 전체 텍스트 생성 (LLM에 보낼 텍스트)
# 라인 단위로 전체 텍스트 생성
full_text = generate_text_from_data(lines_data)

print("="*60)
print("추출된 전체 텍스트 (라인 단위):")
print("="*60)
print(full_text)
print("="*60)
print(f"\n총 {len(full_text)} 글자")


In [ ]:
# Step 5.5: OCR 교정 (LLM을 활용한 오타 수정)
# OCR 결과에 포함된 오류를 LLM으로 자동 교정
LLM_SERVER_URL = "http://gpurent.kogrobo.com:51089/v1"

print("[INFO] OCR 교정 시작...")
print("원본 텍스트에 오류가 있는지 확인하고 수정합니다...\n")

# 전체 텍스트를 LLM으로 교정
corrected_text = call_llm_for_correction(full_text, LLM_SERVER_URL)

if corrected_text != full_text:
    print("="*60)
    print("교정 결과:")
    print("="*60)
    
    # 교정된 텍스트를 라인별로 분할하여 lines_data 업데이트
    corrected_lines = corrected_text.split('\n')
    
    print(f"교정 전 라인 수: {len(lines_data)}")
    print(f"교정 후 라인 수: {len(corrected_lines)}")
    
    # lines_data 업데이트
    for i, line_item in enumerate(lines_data):
        if i < len(corrected_lines):
            original = line_item['text']
            corrected = corrected_lines[i].strip()
            line_item['text'] = corrected
            line_item['corrected'] = (original != corrected)
            
            # 변경된 라인만 표시
            if original != corrected:
                print(f"\n[수정] Line {i+1}:")
                print(f"  원본: {original}")
                print(f"  교정: {corrected}")
        else:
            line_item['corrected'] = False
    
    # 교정된 전체 텍스트 업데이트
    full_text = corrected_text
    
    print("\n" + "="*60)
    print("교정 완료!")
    print("="*60)
else:
    print("[INFO] OCR 교정: 변경사항 없음 (오류가 없거나 감지되지 않음)")
    # 교정되지 않았음을 표시
    for line_item in lines_data:
        line_item['corrected'] = False

print(f"\n교정된 전체 텍스트 (처음 300글자):")
print("-"*60)
print(full_text[:300] + "..." if len(full_text) > 300 else full_text)
print("-"*60)


In [ ]:
# Step 6: LLM 내용 설명 생성
# 교정된 텍스트(full_text)를 사용하여 이미지 내용 설명 생성
# LLM_SERVER_URL은 Step 5.5에서 이미 정의됨

print("[INFO] LLM에 텍스트 전송 중...")
print("교정된 텍스트를 기반으로 이미지 내용을 분석합니다...\n")

description = call_llm_for_description(full_text, LLM_SERVER_URL)

if description:
    print("\n" + "="*60)
    print("LLM 이미지 내용 설명:")
    print("="*60)
    print(description)
    print("="*60)
else:
    print("[WARN] LLM 설명이 생성되지 않았습니다.")


In [ ]:
# Step 7: 결과 저장
TXT_PATH = "output/output.txt"
CSV_PATH = "output/output.csv"

# 라인 단위로 저장
save_output(lines_data, TXT_PATH, CSV_PATH, description)

print(f"[완료] TXT 저장: {TXT_PATH}")
print(f"[완료] CSV 저장: {CSV_PATH}")
print(f"[완료] 저장된 라인 수: {len(lines_data)}")

# 저장된 파일 미리보기
print("\n" + "="*60)
print("저장된 TXT 파일 내용 (미리보기 - 처음 500글자):")
print("="*60)
with open(TXT_PATH, 'r', encoding='utf-8') as f:
    content = f.read()
    print(content[:500] + "..." if len(content) > 500 else content)
    print(f"\n... (총 {len(content)} 글자)")


In [10]:
# PDF 텍스트 추출 테스트
import PyPDF2

pdf_path = "pdf/pdf_test_sample.pdf"

print(f"[INFO] PDF 텍스트 추출 시작: {pdf_path}")
print("="*60)

text_items = []

try:
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        total_pages = len(pdf_reader.pages)
        
        print(f"[INFO] PDF 총 페이지 수: {total_pages}\n")
        
        for page_num, page in enumerate(pdf_reader.pages, start=1):
            try:
                text = page.extract_text()
                if text and text.strip():
                    # 텍스트를 라인 단위로 분리하여 각각 저장
                    lines = text.strip().split('\n')
                    for line in lines:
                        line = line.strip()
                        if line and len(line) > 1:  # 1자 이상만 저장
                            text_items.append({
                                "page_num": page_num,
                                "text": line,
                                "text_type": "pdf_extracted"
                            })
                    print(f"[INFO] 페이지 {page_num}: {len(lines)}개 라인 추출")
                else:
                    print(f"[WARN] 페이지 {page_num}: 텍스트 추출 실패 (스캔된 PDF일 수 있음)")
            except Exception as e:
                print(f"[WARN] 페이지 {page_num} 텍스트 추출 실패: {e}")
                
except Exception as e:
    print(f"[ERROR] PDF 텍스트 추출 실패: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*60)
print(f"[INFO] 총 추출된 텍스트 아이템: {len(text_items)}개")
print("="*60)

# 추출된 텍스트 미리보기
if text_items:
    print("\n[추출된 텍스트 미리보기 - 처음 20개]:")
    print("-"*60)
    for i, item in enumerate(text_items[:20]):
        preview = item['text'][:80] + '...' if len(item['text']) > 80 else item['text']
        print(f"  [{i+1}] Page {item['page_num']}: {preview}")
    
    if len(text_items) > 20:
        print(f"  ... (총 {len(text_items)}개 아이템)")
    
    print("\n[전체 텍스트]:")
    print("-"*60)
    full_text = "\n".join([item['text'] for item in text_items])
    print(full_text)
    print("-"*60)
    print(f"\n총 {len(full_text)} 글자")
else:
    print("\n[WARN] PDF에서 텍스트를 추출할 수 없습니다.")
    print("스캔된 PDF이거나 이미지 기반 PDF일 수 있습니다.")
    print("OCR 결과에만 의존합니다.")

[INFO] PDF 텍스트 추출 시작: pdf/pdf_test_sample.pdf
[INFO] PDF 총 페이지 수: 3

[INFO] 페이지 1: 46개 라인 추출
[INFO] 페이지 2: 31개 라인 추출
[INFO] 페이지 3: 31개 라인 추출

[INFO] 총 추출된 텍스트 아이템: 93개

[추출된 텍스트 미리보기 - 처음 20개]:
------------------------------------------------------------
  [1] Page 1: TCC80 7x-Application Note for Boot Sequence  Rev. 0.10
  [2] Page 1: 8/18 Chapter 3
  [3] Page 1: 3 RESET HIERARCHY
  [4] Page 1: Some applications require a robust system that is not affected by the failure of...
  [5] Page 1: or sub-cluster system hangs and is reset, the SIC system is required to operate ...
  [6] Page 1: provides various types of resets.
  [7] Page 1: Figure 3.1 shows four types of resets and the reset sources of each reset  in th...
  [8] Page 1: different scopes.
  [9] Page 1: Note:  In the Normal Boot without SIC  boot sequence, the SIC sub -system is not...
  [10] Page 1: According to the configuration, the safety -related blocks are set to be selecti...
  [11] Page 1: when reset for SDM is masked,

In [11]:
# PDF 텍스트 추출 결과 확인
# 위 셀에서 추출한 텍스트를 확인할 수 있습니다.

if 'text_items' in locals() and text_items:
    print(f"추출된 텍스트 아이템 수: {len(text_items)}")
    print(f"페이지별 통계:")
    
    from collections import Counter
    page_counts = Counter([item['page_num'] for item in text_items])
    for page_num, count in sorted(page_counts.items()):
        print(f"  페이지 {page_num}: {count}개 라인")
else:
    print("텍스트 추출이 완료되지 않았습니다. 위 셀을 먼저 실행해주세요.")




추출된 텍스트 아이템 수: 93
페이지별 통계:
  페이지 1: 40개 라인
  페이지 2: 27개 라인
  페이지 3: 26개 라인
